In [1]:
%load_ext autoreload
%autoreload 2
from TNR import *

In [2]:
chi={}
chi['w'],chi['y'],chi['u'],chi['v']=9,9,9,9
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

J=1
# htemp=(KP(s0,sx,sx,s0)+KP(s0,sz,sz,s0))+\
#     J*(KP(sx,sx,s0,s0)+KP(sz,sz,s0,s0)+KP(s0,s0,sx,sx)+KP(s0,s0,sz,sz))/2
# htemp=KP(sz,sz)+J*(KP(sx,s0)+KP(s0,sx))/2
# htemp=(KP(sz,s0)+KP(s0,sz))/2+(KP(sx,s0)+KP(s0,sx))/2
# htemp=KP(sx,sx)+KP(sz,sz)
htemp=-KP(R,R.conj())-J*(KP(M,np.eye(3))+KP(np.eye(3),M))/2
htemp+=htemp.T.conj()

d=int(np.sqrt(htemp.shape[0]))
Ainit=expm(-0.002*htemp).reshape(d,d,d,d)
A0=Asplit(Ainit,chi)
for i in range(8):
    A0=coarsen(A0,chi)

# na=np.newaxis
# s=np.array([1,-1])
# Tc=2/np.log(1+np.sqrt(2))
# T=1*Tc
# A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
#                 +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
# A0/=np.linalg.norm(A0)

vertical truncation err: 0.0
horizontal truncation err: 0.0
Aout shape: (9, 9, 9, 9)
Coarsen truncation error: 3.09722e-14
Coarsen truncation error: 2.53113e-12
Coarsen truncation error: 9.57617e-11
Coarsen truncation error: 3.23403e-09
Coarsen truncation error: 1.06877e-07
Coarsen truncation error: 3.42699e-06
Coarsen truncation error: 9.39821e-05
Coarsen truncation error: 0.00163805


In [3]:
RGsteps=5
A_list=[None]*(RGsteps+1)
Anorm_list=[None]*(RGsteps+1)
B_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list=[[None]*RGsteps for i in range(10)]
A_list[0]=A0

for i in range(RGsteps):
    print('RG step: ',i)
    Aold=A_list[i]
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm=doTNR(Aold,chi)
    B_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i],A_list[i+1],Anorm_list[i+1]=\
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm
    print('A.shape= ',A.shape)
    print()

RG step:  0
iter=0, U.Uenv=0.37715
iter=100, U.Uenv=0.377244
iter=200, U.Uenv=0.377244
iter=300, U.Uenv=0.377244
error in U, vl, vr optimization: 2.16746e-05
error in yl, yr, D optimization: 0.00352967
error in w optimization: 0.13533
iter=0, u.uenv=0.459625, v.venv=0.687435
iter=1000, u.uenv=0.997441, v.venv=0.997441
iter=2000, u.uenv=0.997441, v.venv=0.997441
Adiff 0.0715404
A.shape=  (9, 9, 9, 9)

RG step:  1
iter=0, U.Uenv=0.362802
iter=100, U.Uenv=0.363384
iter=200, U.Uenv=0.363384
iter=300, U.Uenv=0.363384
error in U, vl, vr optimization: 0.00295224
error in yl, yr, D optimization: 0.0183735
error in w optimization: 0.107334
iter=0, u.uenv=0.238745, v.venv=0.757757
iter=1000, u.uenv=0.933075, v.venv=0.933075
iter=2000, u.uenv=0.933075, v.venv=0.933075
Adiff 0.365855
A.shape=  (9, 9, 9, 9)

RG step:  2
iter=0, U.Uenv=0.407738
iter=100, U.Uenv=0.4081
iter=200, U.Uenv=0.4081
iter=300, U.Uenv=0.4081
error in U, vl, vr optimization: 0.00652983
error in yl, yr, D optimization: 0.007453

In [4]:
i=3
A=A_list[i]

In [5]:
T=np.einsum(A,[2,1,2,0])
# ew,_=np.linalg.eigh(T)
ew = scipy.sparse.linalg.eigsh(T, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[5]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.        ,  0.1135802 ,  0.11369415,  0.68090692,  0.99453014,
        1.        ,  1.11101356,  1.16994599,  1.53537921])

In [6]:
T=A[0,:,0,:]
ew,_=np.linalg.eigh(T)
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[4]-el[0]))[:100]

array([-0.        ,  0.37644795,  0.37664347,  0.99986256,  1.        ,
        1.19454926,  1.5762499 ,  1.57636964,  1.76231636])

In [7]:
T2=np.einsum(A,[4,2,5,0],A,[5,3,4,1]).reshape(chiw**2,chiw**2)
ew,_=np.linalg.eigh(T2)
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[5]-el[0]))[:100]

array([-0.        ,  0.12382146,  0.12394023,  0.7238654 ,  0.99913258,
        1.        ,  1.04334269,  1.04377795,  1.17747608,  1.17928433,
        1.56064157,  1.6225452 ,  1.69509176,  1.70723895,  1.88038437,
        1.91114696,  1.92194526,  1.92901019,  1.97201497,  1.98163755,
        2.00800223,  2.02865327,  2.0300405 ,  2.03612831,  2.1940424 ,
        2.217409  ,  2.23503289,  2.25872677,  2.53872458,  2.55509233,
        2.57373418,  2.65074544,  2.70418326,  2.82038176,  2.85230575,
        2.85868436,  2.8625467 ,  2.90253073,  2.93444685,  2.94367363,
        2.95341063,  2.95831165,  3.01416566,  3.14303342,  3.14738312,
        3.16814256,  3.17931964,  3.22832093,  3.51291572,  3.54689269,
        3.62903015,  3.68397536,  3.79041836,  3.80900948,  3.83759554,
        3.89619564,  3.91463582,  3.9615623 ,  3.97144734,  4.02310704,
        4.04826791,  4.21530989,  4.22410177,  4.23491066,  4.24028495,
        4.35190882,  4.41434177,  4.72497934,  4.74920396,  4.75

In [9]:
T3=np.einsum(A,[10,3,8,0],A,[8,4,9,1],A,[9,5,10,2]).reshape(chiw**3,chiw**3)
ew = scipy.sparse.linalg.eigsh(T3, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[6]-el[0]))[:100]*1.1333

array([-0.        ,  0.14385892,  0.14398865,  0.81434705,  1.1329059 ,
        1.1329059 ,  1.1333    ,  1.1333    ,  1.3316197 ,  1.33181916,
        1.78391798,  1.78391798,  1.8422797 ,  1.8422797 ,  2.04844586,
        2.04844586,  2.15227212,  2.15227212,  2.15253155,  2.15253155,
        2.15516226,  2.15767979,  2.25761931,  2.25761931,  2.25810131,
        2.25810131,  2.38659225,  2.38659225,  2.38971384,  2.38971384,
        2.77627599,  2.79355953,  2.83309255,  2.88261656,  2.91815017,
        2.91815017,  3.07453871,  3.07453871,  3.09290377,  3.09585805,
        3.12714059,  3.12714059,  3.12850808,  3.12850808,  3.26087729,
        3.26374326,  3.26838713,  3.27087652,  3.27087652,  3.27102716,
        3.27287622,  3.27833999,  3.27833999,  3.33160022,  3.37737924,
        3.38525129,  3.38560134,  3.39733827,  3.40348148,  3.40348148,
        3.4037256 ,  3.4037256 ,  3.41455769,  3.42116735,  3.43291784,
        3.49409649,  3.67012344,  3.67012344,  3.68595554,  3.68

In [ ]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew,_ = np.linalg.eig(T4)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el)[:100]

In [10]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew = scipy.sparse.linalg.eigsh(T4, k=100, which='LM', return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[9]-el[0]))[:100]

array([-0.        ,  0.11120694,  0.11129857,  0.60967463,  0.84514573,
        0.84514573,  0.84540944,  0.84540944,  0.9999294 ,  1.        ,
        1.3393976 ,  1.3393976 ,  1.37175021,  1.37175021,  1.51251104,
        1.512751  ,  1.59378733,  1.59526716,  1.6042686 ,  1.60515118,
        1.60637049,  1.60677266,  1.61527095,  1.61638843,  1.67124098,
        1.67169336,  1.76190208,  1.76190208,  1.76198948,  1.76198948,
        2.07732007,  2.08403442,  2.09454567,  2.11633091,  2.138593  ,
        2.17125073,  2.19894356,  2.19974089,  2.27386227,  2.27386227,
        2.34211511,  2.34211511,  2.34229048,  2.34229048,  2.35874726,
        2.35874726,  2.36138186,  2.36138186,  2.3632677 ,  2.3632677 ,
        2.42905698,  2.42905698,  2.43181136,  2.43181136,  2.44449707,
        2.44449707,  2.44510313,  2.44510313,  2.51365326,  2.51486507,
        2.51584252,  2.51642428,  2.52015858,  2.52438297,  2.55511275,
        2.55511275,  2.55635288,  2.55635288,  2.61222629,  2.61

In [11]:
(el/(el[6]-el[0]))[:100]*(1.1333)

array([-0.        ,  0.14907667,  0.14919951,  0.8172895 ,  1.13294649,
        1.13294649,  1.1333    ,  1.1333    ,  1.34043924,  1.34053388,
        1.79550786,  1.79550786,  1.83887763,  1.83887763,  2.02757229,
        2.02789397,  2.13652591,  2.13850968,  2.15057642,  2.15175953,
        2.15339407,  2.15393318,  2.16532543,  2.16682346,  2.24035515,
        2.24096159,  2.36188943,  2.36188943,  2.36200659,  2.36200659,
        2.78471793,  2.79371875,  2.80780943,  2.83701328,  2.86685637,
        2.91063517,  2.94775834,  2.94882719,  3.04818941,  3.04818941,
        3.13968465,  3.13968465,  3.13991974,  3.13991974,  3.16198062,
        3.16198062,  3.16551238,  3.16551238,  3.16804042,  3.16804042,
        3.25623318,  3.25623318,  3.25992551,  3.25992551,  3.27693114,
        3.27693114,  3.27774358,  3.27774358,  3.36963736,  3.37126183,
        3.37257214,  3.373352  ,  3.37835795,  3.38402089,  3.42521521,
        3.42521521,  3.42687764,  3.42687764,  3.50177784,  3.50

In [12]:
T4reduced=np.einsum(A,[11,0,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,3,11,3]).reshape(chiw**2,chiw**2)
ew = scipy.sparse.linalg.eigsh(T4reduced, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[9]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.00000000e+00,  4.99596599e-09,  9.20580353e-01,  9.23027805e-01,
        9.74246459e-01,  9.91745214e-01,  9.96888023e-01,  9.97677177e-01,
        9.98760396e-01,  1.00000000e+00,  1.03712251e+00,  1.04902022e+00,
        1.07013854e+00,  1.08046930e+00,  1.12311929e+00,  1.17195892e+00,
        1.18890350e+00,  1.24217920e+00,  1.24406100e+00,  1.27621656e+00,
        1.27699451e+00,  1.28690860e+00,  1.28208019e+00,  1.27945905e+00,
        1.22885842e+00,  1.21879832e+00,  1.19262162e+00,  1.10913150e+00,
        1.10420462e+00,  1.08104846e+00,  1.06700641e+00,  1.05314342e+00,
        9.94678778e-01,  9.41076610e-01,  4.05513452e-01,  4.05513447e-01])

In [88]:
#eval mag
tensor_list={}
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list
op=sz
A0,Az=Asplit(Ainit,chi,op=op)
for i in range(8):
    A0,Az=coarsen(A0,chi,Az)
MOq=MOq_(A0,Az)

eval_op(MOq,tensor_list)  

vertical truncation err: 0.0
horizontal truncation err: 0.0
Aout shape: (4, 4, 4, 4)
Coarsen truncation error: 0
Coarsen truncation error: 2.66436e-15
Coarsen truncation error: 2.66384e-15
Coarsen truncation error: 2.66176e-15
Coarsen truncation error: 2.87462e-15
Coarsen truncation error: 8.43189e-14
Coarsen truncation error: 1.45004e-11
Coarsen truncation error: 1.81732e-09
RGsteps: 0 , <O> = (-0.00018785145219859996+0j)
RGsteps: 1 , <O> = (-0.0011401166157475441+0j)
RGsteps: 2 , <O> = (-0.005013877429358866+0j)
RGsteps: 3 , <O> = (-0.020506606462526226+0j)
RGsteps: 4 , <O> = (-0.08230469172582247+0j)
RGsteps: 5 , <O> = (-0.31887115689740103+0j)
RGsteps: 6 , <O> = (-0.8666364493603469+0j)
RGsteps: 7 , <O> = (-0.9986945863610348+0j)
RGsteps: 8 , <O> = (-0.9987447893014204+0j)
RGsteps: 9 , <O> = (-0.998744789301423+0j)
RGsteps: 10 , <O> = (-0.9987447893014197+0j)
RGsteps: 11 , <O> = (-0.9987447893014292+0j)
RGsteps: 12 , <O> = (-0.9987447893014219+0j)
RGsteps: 13 , <O> = (-0.9987447893

(-0.9987447893014207+2.569990312529202e-16j)

In [28]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

0.629648162973007


scaling dimensions

In [24]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]

gl=gl_(vl)
gr=gr_(vr)
gu=gu_(Aold,U,vl,vr)
gnw=gnw_(vl,vr,yr,D,w)
gne=gne_(vl,vr,yl,D,w)
gnw,gne=gauge_(gnw,gne,u,v)

chiytemp=min(chiy,Aold.shape[0])
chiwtemp=min(chiw,Aold.shape[1])
def scaleop(v):
    M=v.reshape(chiwtemp,chiwtemp,chiytemp,chiytemp,chiytemp,chiytemp,chiwtemp,chiwtemp)
    Mscaled=rg_(M,gl,gr,gu,gnw,gne)
    return Mscaled.flatten()

n=chiwtemp**4*chiytemp**4

In [25]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=20
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  1.82548082,  1.84913751,  1.90390264,  1.90861535,
        2.04672911,  2.05618218,  2.79575673,  2.88839787,  3.2273013 ,
        3.29335963,  3.34877207,  3.37503155,  3.55961591,  3.67312588,
        3.74014019,  3.74075378,  3.74605071,  3.81223776,  3.84210499])

In [228]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]


wgauged=w@u
# wgauged=w

def scaleop(v):
    M=v.reshape(chiw,chiw,chiw,chiw)
    mera=np.einsum(M,[0,1,2,3],U,[0,1,8,9],wgauged,[12,8,4],wgauged,[9,13,5],U.conj(),[2,3,10,11],wgauged.conj(),[12,10,6],wgauged.conj(),[11,13,7],optimize=("greedy",2**100))
    return mera.flatten()

n=chiw**4

import scipy
from scipy.sparse.linalg import LinearOperator

N_level=20
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=20,which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.18126637,  0.56473003,  0.7651075 ,  0.82930577,
        0.93717961,  1.15840441,  1.16340123,  1.21039609,  1.23688751,
        1.46377128,  1.54051149,  1.57518613,  1.57518613,  1.64407593,
        1.64407593,  1.64840365,  1.86336301,  1.8636821 ,  1.86368214])

In [11]:
# compress legs

# def wrenv_(A):
#     return np.einsum(A,[5,6,0,8],A,[4,7,1,6],A.conj(),[4,7,3,9],A.conj(),[5,9,2,8])
# def wlenv_(A):
#     return np.einsum(A,[0,6,5,8],A,[1,7,4,6],A.conj(),[3,7,4,9],A.conj(),[2,9,5,8])
# def wvenv_(A):
#     return np.einsum(A,[8,0,6,5],A,[6,1,7,4],A.conj(),[8,2,9,5],A.conj(),[9,3,7,4])
# chik=20
# wrenv=wrenv_(A)
# _,wr=compress(wrenv,chik)
# wlenv=wlenv_(A)
# _,wl=compress(wlenv,chik)
# wvenv=wvenv_(A)
# _,wv=compress(wvenv,chik)

# wrenv=wrenv_(Aold)
# _,wrold=compress(wrenv,chik)
# wlenv=wlenv_(Aold)
# _,wlold=compress(wlenv,chik)
# wvenv=wvenv_(Aold)
# _,wvold=compress(wvenv,chik)

# glw=np.einsum(gl,[3,4,1,2],wrold.conj(),[3,4,0])
# grw=np.einsum(gr,[3,4,1,2],wlold.conj(),[3,4,0])
# guw=np.einsum(gu,[3,4,1,2],wvold.conj(),[3,4,0])

# def scaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4,[8,9,10,11],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten()
# def rscaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4.conj(),[0,1,2,3],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten().conj()

# n=chik**4

In [31]:
MOq.shape

(4, 4, 4, 4, 4, 4, 4, 4)